In [1]:
import os
import shutil
import time

from PIL import Image
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim
from torch.nn.utils import clip_grad_norm

from tqdm import tqdm_notebook

from dataset import TSNDataSet
from models import TSN
from opts import parser
from transforms import *

In [2]:
args_str = """ucf101 RGB \
    /media/e/vsd/data/ucf101_preprocessed/split_01/file_lists/train_rgb.txt \
    /media/e/vsd/data/ucf101_preprocessed/split_01/file_lists/test_rgb.txt \
    --arch BNInception --num_segments 3 \
    --gd 20 --lr 0.001 --lr_steps 30 60 --epochs 80 \
    -b 32 -j 4 \
    --snapshot_pref ucf101_bninception_"""

args = parser.parse_args(args_str.split())

In [2]:
arch = 'BNInception'
batch_size = 4
clip_gradient = 20.0
consensus_type = 'avg'
dataset = 'ucf101'
dropout = 0.5
epochs = 80
eval_freq = 5
evaluate = False
flow_prefix = ''
gpus = None
k = 3
loss_type = 'nll'
lr = 0.001
lr_steps = [30.0, 60.0]
modality = 'RGB'
momentum = 0.9
no_partialbn = False
num_segments = 3
print_freq = 10
resume = False
snapshot_pref = 'ucf101_bninception_'
start_epoch = 0
weight_decay = 0.0005
workers = 4

num_class = 101

checkpoint_path = 'ucf101_bninception__rgb_model_best.pth.tar'

In [3]:
model = TSN(num_class, num_segments, modality,
                base_model=arch,
                consensus_type=consensus_type, dropout=dropout, partial_bn=not no_partialbn)

crop_size = model.crop_size
scale_size = model.scale_size
input_mean = model.input_mean
input_std = model.input_std
policies = model.get_optim_policies()
train_augmentation = model.get_augmentation()

model = torch.nn.DataParallel(model, device_ids=gpus).cuda()


Initializing TSN with base model: BNInception.
TSN Configurations:
    input_modality:     RGB
    num_segments:       3
    new_length:         1
    consensus_module:   avg
    dropout_ratio:      0.5
        


/usr/local/lib/python3.5/dist-packages/torch/nn/modules/module.py:360: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)


In [4]:
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])

In [5]:
# if args.modality != 'RGBDiff':
#     normalize = GroupNormalize(input_mean, input_std)
# else:
#     normalize = IdentityTransform()

# if args.modality == 'RGB':
#     data_length = 1
# elif args.modality in ['Flow', 'RGBDiff']:
#     data_length = 5

data_length = 1
# data_length = 5

normalize = IdentityTransform()

In [9]:
transform = torchvision.transforms.Compose([
                   GroupScale(int(scale_size)),
                   GroupCenterCrop(crop_size),
                   Stack(roll=arch == 'BNInception'),
                   ToTorchFormatTensor(div=arch != 'BNInception'),
                   normalize,
               ])

In [13]:
# val_loader = torch.utils.data.DataLoader(
#     TSNDataSet("", val_list, num_segments=num_segments,
#                new_length=data_length,
#                modality=modality,
#                image_tmpl="{:04d}.jpg" if modality in ['RGB', 'RGBDiff'] else "{:04d}.flo",
#                random_shift=False,
#                transform=torchvision.transforms.Compose([
#                    GroupScale(int(scale_size)),
#                    GroupCenterCrop(crop_size),
#                    Stack(roll=arch == 'BNInception'),
#                    ToTorchFormatTensor(div=arch != 'BNInception'),
#                    normalize,
#                ])),
#     batch_size=1, shuffle=False,
#     num_workers=workers, pin_memory=True)

In [6]:
model.eval()

Freezing BatchNorm2D except the first one.


DataParallel (
  (module): TSN (
    (base_model): BNInception (
      (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv1_relu_7x7): ReLU (inplace)
      (pool1_3x3_s2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
      (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2_relu_3x3_reduce): ReLU (inplace)
      (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
      (conv2_relu_3x3): ReLU (inplace)
      (pool2_3x3_s2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
      (inception_3a_1x1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (inception_3a_1x1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (

In [107]:
def predict_segment(paths):    
    imgs = [Image.open(path).convert('RGB') for path in paths]
    imgs = transform(imgs)
    imgs = imgs.unsqueeze(0)
    
    input_var = torch.autograd.Variable(imgs, volatile=True)

    return model(input_var).cpu().data.numpy()

In [145]:
def softmax(raw_score, T=1):
        exp_s = np.exp((raw_score - raw_score.max(axis=-1)[..., None])*T)
        sum_s = exp_s.sum(axis=-1)
        return exp_s / sum_s[..., None]

def sliding_window_aggregation_func(score, spans=[1, 2, 4, 8, 16], overlap=0.2, norm=True, fps=1):
    """
    This is the aggregation function used for ActivityNet Challenge 2016
    :param score:
    :param spans:
    :param overlap:
    :param norm:
    :param fps:
    :return:
    """
    frm_max = score.max(axis=1)
    slide_score = []

    def top_k_pool(scores, k):
        return np.sort(scores, axis=0)[-k:, :].mean(axis=0)

    for t_span in spans:
        span = t_span * fps
        step = int(np.ceil(span * (1-overlap)))
        local_agg = [frm_max[i: i+span].max(axis=0) for i in range(0, frm_max.shape[0], step)]
        k = max(15, len(local_agg)/4)
        slide_score.append(top_k_pool(np.array(local_agg), k))

    out_score = np.mean(slide_score, axis=0)

    if norm:
        return softmax(out_score)
    else:
        return out_score
        
def eval_video(video_path):
    segment_duration = 5
    fps = 25
    cnt_indexer = 1
    num_frame_per_segment = 48
    frame_cnt = len(os.listdir(video_path))
    segment_length = segment_duration * fps
    
    ticks = np.arange(1, frame_cnt, segment_length)
    
    preds = {}
    
    
    for start, end in zip(ticks, ticks[1:]):
        end = min(int(end + segment_length / 2), frame_cnt)
        idxs = np.sort(np.random.choice(np.arange(start, end), num_frame_per_segment, replace=False))

        paths = [os.path.join(video_path, '{:04d}.jpg'.format(tick)) for tick in idxs]
        scores = predict_segment(paths)
        
        scores = np.expand_dims(scores, 0)

        preds[(start // fps, end // fps)] = sliding_window_aggregation_func(scores)
    
    return preds
    
#     step = int((frame_cnt - stack_depth) / (num_frame_per_video - 1))
#     if step > 0:
#         frame_ticks = list(range(1, min((2 + step * (num_frame_per_video-1)), frame_cnt+1), step))
#     else:
#         frame_ticks = [1] * num_frame_per_video

#     assert(len(frame_ticks) == num_frame_per_video)

#     frame_scores = []
#     for tick_i, tick_j in zip(frame_ticks, frame_ticks[1:]):
#         idxs = np.sort(np.random.choice(np.arange(tick_i, tick_j), 3, replace=False))
#         paths = [os.path.join(video_path, '{:04d}.jpg'.format(tick)) for tick in idxs]
        
#         scores = predict_segment(paths)
#         frame_scores.append(scores)

#     frame_scores = np.array(frame_scores)
#     frame_scores = np.expand_dims(frame_scores, 0)
    
#     return sliding_window_aggregation_func(frame_scores)

In [146]:
true_prefix = '/data/data/imitations_temp/rgb/'
false_prefix = '/data/data/non_violence_temp/rgb/'

data = [
    [1, true_prefix],
    [0, false_prefix]
]

rows = []

for label, prefix in tqdm_notebook(data):
    for video_name in tqdm_notebook(os.listdir(prefix), leave=False):
        video_path = os.path.join(prefix, video_name)
        scores = eval_video(video_path)
        
        rows.append({
            'label': label,
            'video': video_name,
            'scores': scores
        })

In [147]:
def get_top_5(scores):
    top_5_classes = {}
    
    for ts_mark, scores_ in scores.items():
        top_5 = np.argsort(scores_)[::-1][:5]
        top_5_classes[ts_mark] = [class_index[c_id] for c_id in top_5]
        
    return top_5_classes

for row in rows:
    row['top_5'] = get_top_5(row['scores'])

In [148]:
for row in rows:
    print('-' * 40)
    print(row['video'], row['label'])
    
    for (ts1, ts2), top_5 in sorted(row['top_5'].items()):
        print(ts1, ts2, top_5)

----------------------------------------
IMG_0081 1
0 7 ['Skiing', 'ThrowDiscus', 'WallPushups', 'CricketBowling', 'Swing']
5 12 ['Skiing', 'ThrowDiscus', 'Swing', 'WallPushups', 'CricketBowling']
10 17 ['WallPushups', 'PlayingDhol', 'Swing', 'CricketBowling', 'BabyCrawling']
15 21 ['WallPushups', 'CricketBowling', 'MoppingFloor', 'BabyCrawling', 'Skiing']
----------------------------------------
IMG_0080 1
0 7 ['HandstandWalking', 'BabyCrawling', 'CricketBowling', 'Nunchucks', 'SalsaSpin']
5 12 ['Nunchucks', 'SalsaSpin', 'CricketBowling', 'Swing', 'BabyCrawling']
10 17 ['BabyCrawling', 'SalsaSpin', 'MoppingFloor', 'SoccerJuggling', 'TaiChi']
15 22 ['BabyCrawling', 'MoppingFloor', 'WallPushups', 'SalsaSpin', 'TaiChi']
20 27 ['BabyCrawling', 'Swing', 'JugglingBalls', 'MoppingFloor', 'Lunges']
25 32 ['BabyCrawling', 'MoppingFloor', 'CricketBowling', 'RopeClimbing', 'SoccerJuggling']
30 37 ['RopeClimbing', 'Basketball', 'ThrowDiscus', 'Mixing', 'MoppingFloor']
35 42 ['SkateBoarding', 'Rop

In [124]:
true_prefix = '/data/data/imitations_temp/rgb/'
false_prefix = '/data/data/non_violence_temp/rgb/'

val_list = generate_video_list(true_prefix, false_prefix)

ds = TSNDataSet("", 'val_list.txt', num_segments=num_segments,
                   new_length=data_length,
                   modality=modality,
                   image_tmpl="{:04d}.jpg" if modality in ['RGB', 'RGBDiff'] else "{:04d}.flo",
                   random_shift=False,
                   test_mode=True,
                   transform=torchvision.transforms.Compose([
                       GroupScale(int(scale_size)),
                       GroupCenterCrop(crop_size),
                       Stack(roll=arch == 'BNInception'),
                       ToTorchFormatTensor(div=arch != 'BNInception'),
                       normalize,
                   ]))

In [31]:
criterion = torch.nn.CrossEntropyLoss().cuda()

In [32]:
SEGMENT_TIME = 9
SEQ_LENGTH = 25
FPS = 25

def split_frames(frames):
    def chunks(l, n):
        """Yield successive n-sized chunks from l."""
        for i in range(0, len(l), n):
            yield l[i:i + n]

    n_frames = len(frames)
    segment_length = FPS * SEGMENT_TIME

    for i, chunk in enumerate(chunks(frames, segment_length)):
        if len(chunk) != segment_length:
            times = segment_length // len(chunk)
            yield (chunk * times + chunk[:segment_length - len(chunk) * times],
                   i * SEGMENT_TIME, (i + 1) * SEGMENT_TIME)
        else:
            yield chunk, i * SEGMENT_TIME, (i + 1) * SEGMENT_TIME
        
        
def get_rgb_frames(rgb_frames_path):
    import glob
    
    return list(sorted(glob.glob(rgb_frames_path + '/' + '*jpg')))

def load_segment(frames):
    return [Image.open(frame).convert('RGB') for frame in frames]

def make_prediction_on_segment(segment):
    images = load_segment(segment)
    images = transform(images)
    images = images.unsqueeze(0)
    
    input_var = torch.autograd.Variable(images, volatile=True)
    
    output = model(input_var)
    
    _, pred = output.mean(dim=0).topk(5)
    return pred.cpu().data.numpy()

def build_class_index():
    return {i: class_ for i, class_ in enumerate(sorted(os.listdir('/data/data/ucf101/UCF-101/')))}

In [33]:
class_index = build_class_index()

In [34]:
len(class_index)

101

In [35]:
from dataset import TSNDataSet

In [36]:
def generate_video_list(true_prefix, false_prefix):
    def get_files(prefix, label):
        files = []
        for file in sorted(os.listdir(prefix)):
            path = os.path.join(prefix, file)
            num_frames = len(os.listdir(path))
            file_line = '{} {} {}'.format(path, num_frames, label)

            files.append(file_line)
            
        return files
    
    files = get_files(false_prefix, 0) + get_files(true_prefix, 1)
        
    with open('val_list.txt', 'w+') as f:
        f.write('\n'.join(files))
        
    return files

In [37]:
rows = []

for (video, label), video_info in zip(ds, val_list):
    input_var = torch.autograd.Variable(video, volatile=True)
    print(input_var.size())
    output = model(input_var)

    _, top_5_pred = output.mean(dim=0).topk(5)
    probs = output.mean(dim=0)
    top_5_pred = top_5_pred.cpu().data.numpy()
    probs = probs.cpu().data.numpy()
    
    rows.append({
        'label': label,
        'top_5': top_5_pred,
        'probs': probs
    })

torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])
torch.Size([9, 224, 224])


In [104]:
import pandas as pd

In [105]:
df = pd.DataFrame(rows)

In [116]:
TARGET_CLASSES = [
    'BoxingPunchingBag',
     'BoxingSpeedBag',
     'Fencing',
     'GolfSwing',
     'Hammering',
     'PullUps',
     'PushUps',
     'SumoWrestling',
     'Swing',
     'WallPushups',
]

TARGET_CLASSES = list(sorted(TARGET_CLASSES))

reverse_index = {c: i for i, c in class_index.items()}

OTHER_CLASSES = [c for c in sorted(reverse_index) if c not in TARGET_CLASSES]

target_idxs = [reverse_index[c] for c in TARGET_CLASSES]
other_idxs = [reverse_index[c] for c in OTHER_CLASSES]

In [117]:
df['target_probs'] = df['probs'].map(lambda x: x[target_idxs])
df['other_probs'] = df['probs'].map(lambda x: x[other_idxs])

In [118]:
df.target_probs.values[0]

array([  6.70999861,  -2.96474648,   4.11006355,  -2.08382916,
         2.504632  ,  -2.22515535,   0.55434322,  -2.17256522,
        17.27012825,   1.09243584], dtype=float32)

In [58]:
rgb_frames_prefix = '/data/data/imitations_temp/rgb/'

for video_name in tqdm_notebook(os.listdir(rgb_frames_prefix)):
    video_path = os.path.join(rgb_frames_prefix, video_name)
    frames = get_rgb_frames(video_path)
    print('#' * 40)
    print(video_name)
    print('-' * 40)
    preds = {}
    for segment, start_time, end_time in tqdm_notebook(split_frames(frames), leave=False):
        pred = [class_index[int(class_id)] for class_id in make_prediction_on_segment(segment)]
        preds[(start_time, end_time)] = pred
        
        print(start_time, end_time, pred)

########################################
IMG_0081
----------------------------------------


0 9 ['Skiing', 'WallPushups', 'ThrowDiscus', 'Swing', 'RopeClimbing']
9 18 ['WallPushups', 'CricketBowling', 'BabyCrawling', 'Biking', 'RopeClimbing']
18 27 ['WallPushups', 'BabyCrawling', 'SoccerJuggling', 'PlayingDhol', 'ThrowDiscus']
########################################
IMG_0080
----------------------------------------


0 9 ['CricketBowling', 'BabyCrawling', 'SoccerJuggling', 'Swing', 'HandstandWalking']
9 18 ['SalsaSpin', 'SoccerJuggling', 'MoppingFloor', 'BabyCrawling', 'TaiChi']
18 27 ['BabyCrawling', 'MoppingFloor', 'SoccerJuggling', 'WallPushups', 'PizzaTossing']
27 36 ['RopeClimbing', 'SkateBoarding', 'MoppingFloor', 'ThrowDiscus', 'Basketball']
36 45 ['ThrowDiscus', 'SkateBoarding', 'HandstandWalking', 'Basketball', 'MoppingFloor']
45 54 ['TaiChi', 'SkateBoarding', 'MoppingFloor', 'SalsaSpin', 'SoccerJuggling']
54 63 ['MoppingFloor', 'Nunchucks', 'SoccerJuggling', 'WalkingWithDog', 'ThrowDiscus']
########################################
IMG_0085
----------------------------------------


0 9 ['PizzaTossing', 'BodyWeightSquats', 'MoppingFloor', 'PlayingDhol', 'WallPushups']
9 18 ['ShavingBeard', 'PizzaTossing', 'JugglingBalls', 'WallPushups', 'BlowDryHair']
18 27 ['ShavingBeard', 'PizzaTossing', 'Lunges', 'ApplyLipstick', 'TaiChi']
########################################
IMG_0084
----------------------------------------


0 9 ['Swing', 'MoppingFloor', 'SoccerJuggling', 'WallPushups', 'Nunchucks']
9 18 ['Swing', 'Lunges', 'PizzaTossing', 'WallPushups', 'RopeClimbing']
########################################
IMG_0086
----------------------------------------


0 9 ['JugglingBalls', 'WallPushups', 'PizzaTossing', 'SoccerJuggling', 'JumpingJack']
9 18 ['YoYo', 'JugglingBalls', 'Basketball', 'PizzaTossing', 'WallPushups']
18 27 ['WallPushups', 'BoxingPunchingBag', 'Lunges', 'WritingOnBoard', 'PizzaTossing']
########################################
IMG_0083
----------------------------------------


0 9 ['SalsaSpin', 'Lunges', 'MoppingFloor', 'Swing', 'TaiChi']
9 18 ['MoppingFloor', 'TaiChi', 'SkateBoarding', 'BoxingPunchingBag', 'SalsaSpin']
########################################
IMG_0082
----------------------------------------


0 9 ['PizzaTossing', 'BabyCrawling', 'PlayingDhol', 'WallPushups', 'SoccerJuggling']
9 18 ['PizzaTossing', 'Basketball', 'BabyCrawling', 'SoccerJuggling', 'WallPushups']
18 27 ['Basketball', 'RopeClimbing', 'PizzaTossing', 'PlayingDhol', 'BabyCrawling']



In [52]:
rgb_frames_prefix = '/data/data/imitations_temp/rgb/'

for video_name in tqdm_notebook(os.listdir(rgb_frames_prefix)):
    video_path = os.path.join(rgb_frames_prefix, video_name)
    frames = get_rgb_frames(video_path)
    print('#' * 40)
    print(video_name)
    print('-' * 40)
    preds = {}
    for segment, start_time, end_time in tqdm_notebook(split_frames(frames), leave=False):
        pred = [class_index[int(class_id)] for class_id in make_prediction_on_segment(segment)]
        preds[(start_time, end_time)] = pred
        
        print(start_time, end_time, pred)

########################################
wetransfer-892ef3
----------------------------------------


########################################
MVI_2171
----------------------------------------


0 3 ['Lunges', 'HandstandWalking', 'Swing', 'BandMarching', 'MoppingFloor']
3 6 ['Lunges', 'Swing', 'HandstandWalking', 'SkateBoarding', 'BandMarching']
6 9 ['Lunges', 'SkateBoarding', 'Swing', 'HandstandWalking', 'GolfSwing']
9 12 ['Lunges', 'Swing', 'HandstandWalking', 'Basketball', 'SkateBoarding']
12 15 ['Lunges', 'HandstandWalking', 'Swing', 'SkateBoarding', 'Basketball']

15 18 ['Lunges', 'Swing', 'SkateBoarding', 'HandstandWalking', 'Basketball']
18 21 ['Lunges', 'Swing', 'PizzaTossing', 'BandMarching', 'SkateBoarding']
21 24 ['Lunges', 'Swing', 'PizzaTossing', 'SkateBoarding', 'MoppingFloor']
24 27 ['Lunges', 'Swing', 'PizzaTossing', 'HandstandWalking', 'SkateBoarding']
27 30 ['Lunges', 'Swing', 'HandstandWalking', 'PizzaTossing', 'SkateBoarding']
30 33 ['Lunges', 'SkateBoarding', 'BandMarching', 'Swing', 'PizzaTossing']
########################################
MVI_2152
----------------------------------------


0 3 ['Swing', 'Lunges', 'HandstandWalking', 'PoleVault', 'PizzaTossing']
3 6 ['Swing', 'HandstandWalking', 'Lunges', 'PoleVault', 'Fencing']
6 9 ['Swing', 'HandstandWalking', 'Lunges', 'Fencing', 'PoleVault']
9 12 ['Swing', 'HandstandWalking', 'Lunges', 'PizzaTossing', 'PoleVault']
12 15 ['Swing', 'Lunges', 'HandstandWalking', 'PoleVault', 'BenchPress']
15 18 ['Swing', 'Lunges', 'RopeClimbing', 'HandstandWalking', 'PizzaTossing']
18 21 ['Swing', 'HandstandWalking', 'Lunges', 'PizzaTossing', 'MoppingFloor']
21 24 ['Swing', 'HandstandWalking', 'Lunges', 'RopeClimbing', 'MoppingFloor']
24 27 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'PizzaTossing']
27 30 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'PizzaTossing']
########################################
MVI_2166
----------------------------------------


0 3 ['PizzaTossing', 'Haircut', 'BodyWeightSquats', 'HandstandWalking', 'Lunges']
3 6 ['PizzaTossing', 'Haircut', 'HandstandWalking', 'BodyWeightSquats', 'Lunges']
6 9 ['PizzaTossing', 'Haircut', 'HandstandWalking', 'BodyWeightSquats', 'Lunges']
9 12 ['PizzaTossing', 'Haircut', 'BodyWeightSquats', 'HandstandWalking', 'Lunges']
12 15 ['PizzaTossing', 'Haircut', 'HandstandWalking', 'BodyWeightSquats', 'Lunges']
15 18 ['PizzaTossing', 'Haircut', 'HandstandWalking', 'BodyWeightSquats', 'Lunges']
18 21 ['PizzaTossing', 'Haircut', 'HandstandWalking', 'BodyWeightSquats', 'Lunges']
21 24 ['PizzaTossing', 'Haircut', 'BodyWeightSquats', 'HandstandWalking', 'Lunges']
24 27 ['PizzaTossing', 'Haircut', 'HandstandWalking', 'BodyWeightSquats', 'Lunges']
27 30 ['PizzaTossing', 'Haircut', 'BodyWeightSquats', 'Lunges', 'HandstandWalking']
30 33 ['PizzaTossing', 'BodyWeightSquats', 'MoppingFloor', 'Haircut', 'Lunges']
########################################
MVI_2153
-------------------------------------

...

0 3 ['Swing', 'Lunges', 'HandstandWalking', 'MoppingFloor', 'RopeClimbing']
3 6 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'Biking']
6 9 ['Swing', 'Lunges', 'HandstandWalking', 'MoppingFloor', 'SalsaSpin']
9 12 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'SalsaSpin']
12 15 ['Swing', 'Lunges', 'HandstandWalking', 'SalsaSpin', 'MoppingFloor']
15 18 ['Swing', 'Lunges', 'HandstandWalking', 'SalsaSpin', 'MoppingFloor']
18 21 ['Swing', 'Lunges', 'HandstandWalking', 'BabyCrawling', 'MoppingFloor']
21 24 ['Swing', 'Lunges', 'HandstandWalking', 'BabyCrawling', 'MoppingFloor']
24 27 ['Swing', 'Lunges', 'HandstandWalking', 'BabyCrawling', 'MoppingFloor']
27 30 ['Swing', 'HandstandWalking', 'Lunges', 'BabyCrawling', 'MoppingFloor']
30 33 ['Swing', 'Lunges', 'HandstandWalking', 'BabyCrawling', 'MoppingFloor']
33 36 ['Swing', 'Lunges', 'HandstandWalking', 'SalsaSpin', 'BabyCrawling']
36 39 ['HandstandWalking', 'Swing', 'Lunges', 'SalsaSpin', 'MoppingFloor']
#################

0 3 ['Lunges', 'HandstandWalking', 'SkateBoarding', 'RopeClimbing', 'BandMarching']
3 6 ['Lunges', 'HandstandWalking', 'SkateBoarding', 'Swing', 'MoppingFloor']
6 9 ['Lunges', 'PizzaTossing', 'HandstandWalking', 'SkateBoarding', 'Swing']
9 12 ['Lunges', 'SkateBoarding', 'HandstandWalking', 'BandMarching', 'MoppingFloor']
12 15 ['Lunges', 'BandMarching', 'Swing', 'MoppingFloor', 'SkateBoarding']
15 18 ['Lunges', 'SkateBoarding', 'BandMarching', 'Swing', 'PizzaTossing']
18 21 ['Lunges', 'HandstandWalking', 'SkateBoarding', 'RopeClimbing', 'MoppingFloor']
21 24 ['Lunges', 'HandstandWalking', 'SkateBoarding', 'MoppingFloor', 'Basketball']
24 27 ['Lunges', 'SkateBoarding', 'HandstandWalking', 'Basketball', 'PizzaTossing']
27 30 ['Lunges', 'SkateBoarding', 'HandstandWalking', 'BandMarching', 'PizzaTossing']
30 33 ['Lunges', 'HandstandWalking', 'SkateBoarding', 'MoppingFloor', 'Biking']
33 36 ['Lunges', 'HandstandWalking', 'SkateBoarding', 'Basketball', 'Swing']
36 39 ['Lunges', 'HandstandWal

0 3 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'SalsaSpin']
3 6 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'CricketBowling']
6 9 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
9 12 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
12 15 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
15 18 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
18 21 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
21 24 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
24 27 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
27 30 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
30 33 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
33 36 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
36 39 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawl

0 3 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
3 6 ['Swing', 'HandstandWalking', 'Lunges', 'BabyCrawling', 'MoppingFloor']
6 9 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'Biking']
9 12 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'BabyCrawling']
12 15 ['Swing', 'HandstandWalking', 'Lunges', 'PoleVault', 'MoppingFloor']
15 18 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'Biking']
18 21 ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'Diving']
21 24 ['Lunges', 'Swing', 'HandstandWalking', 'MoppingFloor', 'Fencing']
24 27 ['Swing', 'Lunges', 'HandstandWalking', 'BenchPress', 'Diving']
27 30 ['Swing', 'Lunges', 'HandstandWalking', 'MoppingFloor', 'Archery']
30 33 ['Swing', 'RopeClimbing', 'HandstandWalking', 'Diving', 'JumpRope']
33 36 ['Lunges', 'Basketball', 'RopeClimbing', 'BandMarching', 'PizzaTossing']
36 39 ['BenchPress', 'MoppingFloor', 'PoleVault', 'Drumming', 'RopeClimbing']
39 42 ['BenchPress', 'PushUps', 'Po

0 3 ['Lunges', 'BandMarching', 'Swing', 'SkateBoarding', 'HandstandWalking']
3 6 ['Lunges', 'SkateBoarding', 'MoppingFloor', 'HandstandWalking', 'BandMarching']
6 9 ['Lunges', 'HandstandWalking', 'SkateBoarding', 'PizzaTossing', 'MoppingFloor']
9 12 ['Lunges', 'SkateBoarding', 'HandstandWalking', 'Basketball', 'PizzaTossing']
12 15 ['Lunges', 'BandMarching', 'SkateBoarding', 'Basketball', 'Swing']
15 18 ['Lunges', 'SkateBoarding', 'Basketball', 'Swing', 'BandMarching']
18 21 ['Lunges', 'BandMarching', 'Swing', 'MoppingFloor', 'HandstandWalking']
21 24 ['Lunges', 'Basketball', 'Swing', 'BandMarching', 'SkateBoarding']
24 27 ['Lunges', 'Swing', 'Basketball', 'SkateBoarding', 'MoppingFloor']
27 30 ['Lunges', 'Swing', 'Basketball', 'SkateBoarding', 'PizzaTossing']
30 33 ['Lunges', 'Basketball', 'SkateBoarding', 'Swing', 'MoppingFloor']
33 36 ['Lunges', 'SkateBoarding', 'PizzaTossing', 'Swing', 'MoppingFloor']
36 39 ['Lunges', 'Basketball', 'SkateBoarding', 'BandMarching', 'PizzaTossing']



In [48]:
preds

{(0, 3): ['Swing',
  'Lunges',
  'HandstandWalking',
  'MoppingFloor',
  'RopeClimbing'],
 (3, 6): ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'Biking'],
 (6, 9): ['Swing', 'Lunges', 'HandstandWalking', 'MoppingFloor', 'SalsaSpin'],
 (9, 12): ['Swing', 'HandstandWalking', 'Lunges', 'MoppingFloor', 'SalsaSpin'],
 (12, 15): ['Swing',
  'Lunges',
  'HandstandWalking',
  'SalsaSpin',
  'MoppingFloor'],
 (15, 18): ['Swing',
  'Lunges',
  'HandstandWalking',
  'SalsaSpin',
  'MoppingFloor'],
 (18, 21): ['Swing',
  'Lunges',
  'HandstandWalking',
  'BabyCrawling',
  'MoppingFloor'],
 (21, 24): ['Swing',
  'Lunges',
  'HandstandWalking',
  'BabyCrawling',
  'MoppingFloor'],
 (24, 27): ['Swing',
  'Lunges',
  'HandstandWalking',
  'BabyCrawling',
  'MoppingFloor'],
 (27, 30): ['Swing',
  'HandstandWalking',
  'Lunges',
  'BabyCrawling',
  'MoppingFloor'],
 (30, 33): ['Swing',
  'Lunges',
  'HandstandWalking',
  'BabyCrawling',
  'MoppingFloor'],
 (33, 36): ['Swing',
  'Lunges',
  'H